In [1]:
import xarray as xr

In [4]:
direct = '/RAID/datasets/f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004/archive/atm/hist/'

# 20yr water tagging experiments (cam only)
CASES = [
    direct+'f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004.cam.h0.210501-212412.nc'
]

In [6]:
ds = xr.open_dataset('/RAID/datasets/f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004/archive/atm/hist/f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004.cam.h0.210501-212412.nc')

In [7]:
ds

<xarray.Dataset>
Dimensions:          (lev: 30, ilev: 31, time: 240, lat: 96, lon: 144,
                      slat: 95, slon: 144, nbnd: 2)
Coordinates:
  * lev              (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev             (ilev) float64 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * time             (time) object 2105-02-01 00:00:00 ... 2125-01-01 00:00:00
  * lat              (lat) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * lon              (lon) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * slat             (slat) float64 -89.05 -87.16 -85.26 ... 85.26 87.16 89.05
  * slon             (slon) float64 -1.25 1.25 3.75 6.25 ... 351.2 353.8 356.2
Dimensions without coordinates: nbnd
Data variables: (12/1095)
    hyam             (lev) float64 ...
    hybm             (lev) float64 ...
    hyai             (ilev) float64 ...
    hybi             (ilev) float64 ...
    P0               float64 ...
    date             (time) int32 ...
    ...               ...
    pom_a1_SRF       (time, lat, lon) float32 ...
    so4_a1_SRF       (time, lat, lon) float32 ...
    so4_a2_SRF       (time, lat, lon) float32 ...
    so4_a3_SRF       (time, lat, lon) float32 ...
    soa_a1_SRF       (time, lat, lon) float32 ...
    soa_a2_SRF       (time, lat, lon) float32 ...
Attributes:
    Conventions:      CF-1.0
    source:           CAM
    case:             f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004
    title:            UNSET
    logname:          aflaim
    host:             dec1057
    Version:          $Name$
    revision_Id:      $Id$
    initial_file:     b.ie12.BRCP85C5CN.f19_g16.LME.004.cam.i.2100-01-01-0000...
    topography_file:  /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/consisten...
    history:          Fri Aug 22 07:32:34 2025: ncrcat f.ie12.BRCP85C5CN.f19_...
    NCO:              netCDF Operators version 5.2.4 (Homepage = http://nco.s...

In [ ]:
ds = xr.open_dataset('/RAID/datasets/f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004/archive/atm/hist/f.ie12.BRCP85C5CN.f19_g16.LME.004_2100watertags.004.cam.h0.210501-212412.nc')